# Dependencies 

In [1]:
import pandas as pd
import re
import sys

from typing import List
from catboost import CatBoostClassifier
from collections import namedtuple, Counter
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import RandomOverSampler
from category_encoders.woe import WOEEncoder
from category_encoders.cat_boost import CatBoostEncoder
from category_encoders.helmert import HelmertEncoder
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score

sys.path.append('..')

from customer_segmentation.constants import PATH_MODELS, RANDOM_STATE, PATH_SUBMISSIONS
from customer_segmentation.experiment_tracking import Tracking
from customer_segmentation import (read_demographic_data,
                                   cat_features_fillna,
                                   compute_metrics,
                                   load_catboost_model,
                                   load_pipeline,
                                   serialize_object_load,
                                   new_experiment,
                                   new_run,
                                   apply_runs_to_experiment,
                                   preprocessing_baseline,
                                   show_metrics_baseline,
                                   kaggle_submission,
                                   n_best_models_from_experiments,
                                   load_trained_model)

# Notebook Specific Helper Functions 

In [2]:
def reformat_data_to_model(df: pd.DataFrame,
                           features_used: List[str],
                           dtypes: pd.Series = None) -> pd.DataFrame:
    """Apply `features used` to `df`, fill the NA values with function
    `cat_features fillna` and apply `dtypes` when necessary
    """
    df_copy = df[features_used].copy()

    cat_features = df_copy.select_dtypes(include=['category', 'object']).columns
    
    df_copy_transformed = cat_features_fillna(
        df_copy,
        cat_features=cat_features
    )
    
    if dtypes is not None:
        df_copy_transformed = df_copy_transformed.astype(dtypes)
    
    return df_copy_transformed


def show_sample(df: pd.DataFrame, n: int = 10) -> None:
    """Displays a sample of `n`, with all columns"""
    with pd.option_context('display.max_columns', None):
        display(df.sample(n).style.hide_index().highlight_null())


def adjust_column_names_for_feature_encoding(columns: List[str]):
    """Correct the column names to avoid collisions
    by adding a _number as a suffix.
    
    Translate numbers into letters, with A being 0,
    B being 1 and so on.
    
    chr(65) -> A
    ord(A) -> 65
    """
    return [column[:-1] + chr(int(re.search('_(\\d)$', column).group(1)) + 65) if re.search('_\\d$', column)
            else column
            for column in columns]

# Constants 

In [3]:
TARGET = 'RESPONSE'
TARGET_CUSTOMER = 'is_customer'

# Read Train And Test Data 

In [4]:
df_mailout_train = read_demographic_data('mailout_train.csv')
df_mailout_test = read_demographic_data('mailout_test.csv')

# Read Customers And German Populations Samples (20%) 

In [5]:
df_customers = read_demographic_data('demographic_data_customers_sample.csv')
df_german_population = read_demographic_data('demographic_data_german_population_sample.csv')

# Load Saved  Models

When we show metrics in local there were different of the ones in `kaggle`

This can be because we are training in different sets and it would be good to check if metrics in training set behaves like the submissions  

In [6]:
catboost_first_model = load_catboost_model('catboost_first_model.cbm')
catboost_less_features_model = load_catboost_model('catboost_less_features_model.cbm')
catboost_with_clusters_model = load_catboost_model('catboost_with_clusters_model.cbm')

X_train_dtypes = serialize_object_load('X_train_dtypes.pkl')
pipeline_kmeans = load_pipeline('pipeline_kmeans.joblib')

# Reformat Data For Each Model

In [7]:
df_mailout_train_first_model_transformed = reformat_data_to_model(
    df_mailout_train,
    features_used=catboost_first_model.feature_names_
)

df_mailout_train_less_features_model_transformed = reformat_data_to_model(
    df_mailout_train,
    features_used=catboost_less_features_model.feature_names_
)

df_mailout_train_with_clusters_transformed = reformat_data_to_model(
    df_mailout_train,
    features_used=catboost_first_model.feature_names_,
    dtypes=X_train_dtypes
)

cluster_colnames = ['cluster_0', 'custer_1', 'cluster_2']

df_mailout_train_with_clusters_transformed[cluster_colnames] = (
    pd.DataFrame(pipeline_kmeans.transform(df_mailout_train_with_clusters_transformed))
)

# Apply Models 

In [8]:
df_mailout_train['score_first_model'] = (
    catboost_first_model
    .predict_proba(df_mailout_train_first_model_transformed)[:, 1]
)

df_mailout_train['score_less_features_model'] = (
    catboost_less_features_model
    .predict_proba(df_mailout_train_less_features_model_transformed)[:, 1]
)

df_mailout_train['score_with_clusters_model'] = (
    catboost_with_clusters_model
    .predict_proba(df_mailout_train_with_clusters_transformed)[:, 1]
)

# Metrics For Each Model 

In [9]:
y_true = df_mailout_train['RESPONSE'].fillna(0)

auc_first_model = roc_auc_score(y_true, df_mailout_train["score_first_model"])
auc_less_features_model = roc_auc_score(y_true, df_mailout_train["score_less_features_model"])
auc_with_clusters_model = roc_auc_score(y_true, df_mailout_train["score_with_clusters_model"])

print(f'AUC First Model: {auc_first_model}')
print(f'AUC Less Features Model: {auc_less_features_model}')
print(f'AUC With Clusters Model: {auc_with_clusters_model}')

AUC First Model: 0.7533230096806949
AUC Less Features Model: 0.7520571254910786
AUC With Clusters Model: 0.749795284227538


# Combining The Models

Combining both scopres from `first_model` and `less_features_model` it results in a better model 

In [10]:
auc_combining_models = roc_auc_score(
    y_true, ((df_mailout_train['score_first_model'] + df_mailout_train['score_less_features_model'])
             / 2)
)

print(f'AUC Combining Models: {auc_combining_models}')

AUC Combining Models: 0.753635753890973


# So We Have A Baseline Of 0.75

We will use the columns used by the first model because them are all of the possible used columns since it eliminates the ID variable and some features that are not present when predicting

In [11]:
df_mailout_train_base = (
    df_mailout_train.loc[:, catboost_first_model.feature_names_ + [TARGET]].fillna(value={TARGET: 0})
)

df_mailout_test_base = df_mailout_test.loc[:, catboost_first_model.feature_names_]

# Training Data Sample 

In [12]:
show_sample(df_mailout_train_base)

# Testing Data Sample 

In [13]:
show_sample(df_mailout_test_base)

# Machine Learning Algorithms

We will use a bunch of algorithms, hyperparameters and feature and data selection

## MLflow Experiment Tracking

We will track the differet runs divided them in 3 main experiments

- Training with customers vs German population data
- Training with train-specific data
- Training with both approaches described above

For each of the experiments we will try different hyperparameters, categorical encoding and feature selection

When we´ve got the best model, we will create a ML project for made the model easy accessible by anyone from source code or directly from github

We will use two of the three parts of `MLflow`, tracking and projects

![MLflow](https://secure.meetupstatic.com/photos/event/9/4/7/a/600_483818010.jpeg)

#  Create The Experiments

In [14]:
experiment_id_customers_vs_german_population = new_experiment(name='Customers-vs-German-Population')
experiment_id_training_data_provided = new_experiment(name='Training-Data-Provided')
experiment_id_mixed_data = new_experiment(name='Mixed-Data')

# First Experiment

We already developed all of the models needed for this experiment, do we only have to track them

In [15]:
zip_run_info = (
    zip(['All possible features', 'Less features', 'With clustering'],
        [catboost_first_model, catboost_less_features_model, catboost_with_clusters_model],
        ['catboost_first_model', 'catboost_less_features_model', 'catboost_with_clusters_model'],
        [auc_first_model, auc_less_features_model, auc_with_clusters_model])
)

Trackings = [
    Tracking(
        run_name=run_name,
        tags=dict(
            model_name=name,
            target=TARGET_CUSTOMER,
            num_features_used=len(model.feature_names_),
            top_feature_importances=(
                Counter(
                    dict(
                        zip(model.feature_names_,
                            model.get_feature_importance())
                    )
                ).most_common(3)
            )
        ),
        params=model.get_params(),
        metrics=dict(AUC=auc),
        model=model,
        model_name=name
    )
    for run_name, model, name, auc in zip_run_info
]


run_ids_customers_vs_german_population = (
    apply_runs_to_experiment(experiment_id_customers_vs_german_population,
                             Trackings)
)

# Second Experiment 

## First Run

We will train with `train-specific data`, there are few examples and it´s probably to perform worse than previous approach, but maybe not because this is the correct target, consisting in a mailout campaign 

`HelmertEncoder` encoder works very well when categorical features are ordinal (looking at the feature attributes file this is our case) but at this particular case it gets a poor perform, because the bad point about this encoder is that it creates a lot of new columns and that is a problem because of the cardinality of our features and the number of them

We will also test `CatBoostEncoder` and `WOEEncoder`, usually cboth perform very well when encoding categorical features

Oversampling to positive class will be done because we have very few examples and the classes are highly unbalanced

In [16]:
df_mailout_train_base_renamed = df_mailout_train_base.copy()

df_mailout_train_base_renamed.columns = adjust_column_names_for_feature_encoding(
    df_mailout_train_base_renamed.columns
)

cat_features = df_mailout_train_base_renamed.select_dtypes(include=['category', 'object']).columns

cat_features_numerical = [
    column
    for column in cat_features
    if df_mailout_train_base_renamed[column].dropna().str.match('^\\d+$').all()
]

features, labels = preprocessing_baseline(df_mailout_train_base_renamed,
                                          cat_features=set(cat_features).difference(cat_features_numerical),
                                          target=TARGET,
                                          test_size=.15,
                                          valid_size=0)

X_train, X_test, _ = features
y_train, y_test, _ = labels

params_xgb = dict(n_estimators=15,
                  max_depth=3,
                  colsample_bytree=.5,
                  min_child_weight=20,
                  gamma=30,
                  random_state=RANDOM_STATE,
                  n_jobs=-1)

params_catboost = dict(num_trees=15,
                       max_depth=3,
                       min_child_samples=25,
                       one_hot_max_size=4,
                       cat_features=cat_features,
                       random_state=RANDOM_STATE,
                       eval_metric='AUC',
                       verbose=0)

pipeline_xgb_oversampling_woe = make_pipeline(
    RandomOverSampler(sampling_strategy=.02, random_state=RANDOM_STATE),
    WOEEncoder(cols=cat_features_numerical),
    CatBoostEncoder(cols=set(cat_features).difference(cat_features_numerical)),
    XGBClassifier(**params_xgb)
)

pipeline_catboost_oversampling = make_pipeline(
    RandomOverSampler(sampling_strategy=.02, random_state=RANDOM_STATE),
    CatBoostClassifier(**params_catboost)
)

X_train_catboost = cat_features_fillna(X_train, cat_features_numerical)

pipeline_xgb_oversampling_woe.fit(X_train, y_train)
pipeline_catboost_oversampling.fit(X_train_catboost, y_train);

# Cross Validation

4-fold cross validation to be sure about the AUC metric

In [17]:
cross_val_score_xgb_oversampling_woe = cross_val_score(
    pipeline_xgb_oversampling_woe,
    X_train,
    y_train,
    cv=StratifiedKFold(n_splits=4, shuffle=True, random_state=RANDOM_STATE),
    scoring='roc_auc',
    n_jobs=-1
)

cross_val_score_catboost_oversampling = cross_val_score(
    pipeline_catboost_oversampling,
    X_train_catboost,
    y_train,
    cv=StratifiedKFold(n_splits=4, shuffle=True, random_state=RANDOM_STATE),
    scoring='roc_auc',
    n_jobs=-1
)

print(f'{cross_val_score_xgb_oversampling_woe.mean()}±{cross_val_score_xgb_oversampling_woe.std()}')
print(f'{cross_val_score_catboost_oversampling.mean()}±{cross_val_score_catboost_oversampling.std()}')

0.686208787087107±0.03104046105766505
0.7443360451574116±0.013728413546340953


# Log Experiment Results 

In [18]:
auc_xgb_oversampling_woe = roc_auc_score(
    y_test, pipeline_xgb_oversampling_woe.predict_proba(X_test)[:, 1]
)

auc_catboost_oversampling = roc_auc_score(
    y_test, pipeline_catboost_oversampling.predict_proba(cat_features_fillna(X_test,
                                                                             cat_features_numerical))[:, 1]
)

zip_run_info = (
    zip(['Oversamplig, WoE and CatBoost encoders with XGBoost', 'Oversampling with CatBoost'],
        [pipeline_xgb_oversampling_woe, pipeline_catboost_oversampling],
        ['pipeline_xgb_oversampling_woe', 'pipeline_catboost_oversampling'],
        [auc_xgb_oversampling_woe, auc_catboost_oversampling])
)

Trackings = [
    Tracking(
        run_name=run_name,
        tags=dict(
            model_name=name,
            target=TARGET,
            num_features_used=len(model.steps[-1][1].feature_names_
                                  if name == 'pipeline_catboost_oversampling'
                                  else model.steps[-2][1].get_feature_names()),
            top_feature_importances=(
                Counter(
                    dict(
                        zip(model.steps[-1][1].feature_names_
                            if name == 'pipeline_catboost_oversampling'
                            else model.steps[-2][1].get_feature_names(),
                            model.steps[-1][1].feature_importances_)
                    )
                ).most_common(3)
            )
        ),
        params={param: value
                for param, value in (pipeline_catboost_oversampling
                                     .steps[-1][1]
                                     .get_params()
                                     .items())
                if param != 'cat_features'},
        metrics=dict(AUC=auc),
        model=model,
        model_name=name
    )
    for run_name, model, name, auc in zip_run_info
]


run_ids_training_data_provided = (
    apply_runs_to_experiment(experiment_id_training_data_provided,
                             Trackings)
)

# Kaggle Submission

![Kaggle CatBoost Oversampling](../images/kaggle_catboost_oversampling.png)

In [19]:
mailout_test_lnr = df_mailout_test['LNR']

df_mailout_test_base_renamed = df_mailout_test_base.copy()

df_mailout_test_base_renamed.columns = adjust_column_names_for_feature_encoding(
    df_mailout_test_base_renamed.columns
)

df_mailout_test_base_renamed_cat_fillna = cat_features_fillna(
    df_mailout_test_base_renamed,
    cat_features=cat_features
)

kaggle_submission(
    column_lnr=mailout_test_lnr,
    y_pred=pipeline_catboost_oversampling.predict_proba(df_mailout_test_base_renamed_cat_fillna)[:, 1],
    submission_filename='catboost_oversampling',
   submission_message='Oversampling catboost trained with customer training data [75/15/0]'
)

100%|██████████| 1.09M/1.09M [00:06<00:00, 181kB/s] 


Successfully submitted to Udacity+Arvato: Identify Customer Segments


## Second Run

We will train again with `train-specific data` but now with only the most important features got in previous executions

Now as we have less features, we are goiong to test `HelmertEncoder`

In [20]:
feature_importances_catboost_oversampling = dict(
    zip(pipeline_catboost_oversampling.steps[-1][1].feature_names_,
        pipeline_catboost_oversampling.steps[-1][1].feature_importances_)
)


feature_importances_catboost_oversampling_relevant = [
    feature
    for feature, importance in feature_importances_catboost_oversampling.items()
    if importance > .01
]

params_catboost = dict(num_trees=30,
                       max_depth=3,
                       min_child_samples=25,
                       one_hot_max_size=4,
                       cat_features=set(cat_features).intersection(feature_importances_catboost_oversampling_relevant),
                       random_state=RANDOM_STATE,
                       eval_metric='AUC',
                       verbose=0)

pipeline_xgb_oversampling_helmert_less_features = make_pipeline(
    RandomOverSampler(sampling_strategy=.02, random_state=RANDOM_STATE),
    HelmertEncoder(cols=set(cat_features).intersection(feature_importances_catboost_oversampling_relevant)),
    XGBClassifier(**params_xgb)
)

pipeline_catboost_oversampling_less_features = make_pipeline(
    RandomOverSampler(sampling_strategy=.02, random_state=RANDOM_STATE),
    CatBoostClassifier(**params_catboost)
)

pipeline_xgb_oversampling_helmert_less_features.fit(
    X_train[feature_importances_catboost_oversampling_relevant],
    y_train)

pipeline_catboost_oversampling_less_features.fit(
    X_train_catboost[feature_importances_catboost_oversampling_relevant], 
    y_train);

# Cross Validation

Again we run 4-fold cross validation to be sure about the AUC metric

In [21]:
cross_val_score_xgb_oversampling_helmert_less_features = cross_val_score(
    pipeline_xgb_oversampling_helmert_less_features,
    X_train[feature_importances_catboost_oversampling_relevant],
    y_train,
    cv=StratifiedKFold(n_splits=4, shuffle=True, random_state=RANDOM_STATE),
    scoring='roc_auc',
    n_jobs=-1
)

cross_val_score_catboost_oversampling_less_features = cross_val_score(
    pipeline_catboost_oversampling_less_features,
    X_train_catboost[feature_importances_catboost_oversampling_relevant],
    y_train,
    cv=StratifiedKFold(n_splits=4, shuffle=True, random_state=RANDOM_STATE),
    scoring='roc_auc',
    n_jobs=-1
)

print(f'{cross_val_score_xgb_oversampling_helmert_less_features.mean()}'
      f'±{cross_val_score_xgb_oversampling_helmert_less_features.std()}')

print(f'{cross_val_score_catboost_oversampling_less_features.mean()}'
      f'±{cross_val_score_catboost_oversampling_less_features.std()}')

0.7477824217935833±0.023980377908516193
0.7577927037640589±0.012358095436621712


# Log Experiment Results 

In [22]:
auc_xgb_oversampling_helmert_less_features = roc_auc_score(
    y_test,
    pipeline_xgb_oversampling_helmert_less_features
    .predict_proba(X_test[feature_importances_catboost_oversampling_relevant])[:, 1]
)

auc_catboost_oversampling_less_features = roc_auc_score(
    y_test,
    pipeline_catboost_oversampling_less_features
    .predict_proba(
        cat_features_fillna(X_test[feature_importances_catboost_oversampling_relevant],
                            set(cat_features).intersection(feature_importances_catboost_oversampling_relevant))
    )[:, 1]
)

zip_run_info = (
    zip(['Less features, oversamplig, Hermert encoder with XGBoost', 'Less features, oversampling with CatBoost'],
        [pipeline_xgb_oversampling_helmert_less_features, pipeline_catboost_oversampling_less_features],
        ['pipeline_xgb_oversampling_helmert_less_features', 'pipeline_catboost_oversampling_less_features'],
        [auc_xgb_oversampling_helmert_less_features, auc_catboost_oversampling_less_features])
)

Trackings = [
    Tracking(
        run_name=run_name,
        tags=dict(
            model_name=name,
            target=TARGET,
            num_features_used=len(model.steps[-1][1].feature_names_
                                  if name == 'pipeline_catboost_oversampling_less_features'
                                  else model.steps[-2][1].get_feature_names()),
            top_feature_importances=(
                Counter(
                    dict(
                        zip(model.steps[-1][1].feature_names_
                            if name == 'pipeline_catboost_oversampling_less_features'
                            else model.steps[-2][1].get_feature_names(),
                            model.steps[-1][1].feature_importances_)
                    )
                ).most_common(3)
            )
        ),
        params={param: value
                for param, value in (pipeline_catboost_oversampling
                                     .steps[-1][1]
                                     .get_params()
                                     .items())
                if param != 'cat_features'},
        metrics=dict(AUC=auc),
        model=model,
        model_name=name
    )
    for run_name, model, name, auc in zip_run_info
]


run_ids_training_data_provided = (
    run_ids_training_data_provided
    + apply_runs_to_experiment(experiment_id_training_data_provided,
                               Trackings)
)

# Kaggle Submission

In [23]:
mailout_test_lnr = df_mailout_test['LNR']

df_mailout_test_base_renamed_less_features = df_mailout_test_base.copy()

df_mailout_test_base_renamed_less_features.columns = adjust_column_names_for_feature_encoding(
    df_mailout_test_base_renamed_less_features.columns
)

df_mailout_test_base_renamed_less_features_cat_fillna = cat_features_fillna(
    df_mailout_test_base_renamed_less_features,
    cat_features=set(cat_features).intersection(feature_importances_catboost_oversampling_relevant)
)

kaggle_submission(
    column_lnr=mailout_test_lnr,
    y_pred=pipeline_catboost_oversampling_less_features.predict_proba(
        df_mailout_test_base_renamed_less_features_cat_fillna[feature_importances_catboost_oversampling_relevant]
    )[:, 1],
    submission_filename='catboost_oversampling_less_features',
    submission_message='Oversampling catboost trained with columns subset of customer training data [75/15/0]'
)

100%|██████████| 1.09M/1.09M [00:06<00:00, 189kB/s]


Successfully submitted to Udacity+Arvato: Identify Customer Segments


# Third Experiment

We will train with all data Arvato provided us

Since the catboost has always worked better for us than the other algorithms we are going to train catboost with different configurations

First we have to join all data from both data sources

**Note**: Just because this is the last part of the project we can modify the column names of customers and German population because it's not used anywhere else

In [24]:
df_customers.columns = adjust_column_names_for_feature_encoding(
    df_customers.columns
)

df_german_population.columns = adjust_column_names_for_feature_encoding(
    df_german_population.columns
)

df_training_all = pd.concat(
    [df_customers[feature_importances_catboost_oversampling_relevant].assign(RESPONSE=1),
     df_german_population[feature_importances_catboost_oversampling_relevant].assign(RESPONSE=0),
     df_mailout_train_base_renamed[feature_importances_catboost_oversampling_relevant + [TARGET]]]
)

cat_features = df_training_all.select_dtypes(include=['category', 'object']).columns

features, labels = preprocessing_baseline(df_training_all,
                                          cat_features=cat_features,
                                          target=TARGET,
                                          test_size=0,
                                          valid_size=0)

X_train, _, _ = features
y_train, _, _ = labels

class_weights = (1, sum(y_train == 0) / sum(y_train == 1))

params_catboost = dict(num_trees=100,
                       max_depth=3,
                       min_child_samples=25,
                       one_hot_max_size=4,
                       class_weights=class_weights,
                       cat_features=cat_features,
                       random_state=RANDOM_STATE,
                       eval_metric='AUC',
                       verbose=0)

catboost_all_data_100 = CatBoostClassifier(**params_catboost)

params_catboost['num_trees'] = 40
catboost_all_data_40 = CatBoostClassifier(**params_catboost)

params_catboost['num_trees'] = 20
catboost_all_data_20 = CatBoostClassifier(**params_catboost)

catboost_all_data_100.fit(X_train, y_train, verbose=False)
catboost_all_data_40.fit(X_train, y_train, verbose=False)
catboost_all_data_20.fit(X_train, y_train, verbose=False)

# Cross Validation

Again we run 4-fold cross validation to be sure about the AUC metric

In [25]:
cross_val_score_catboost_all_data_100 = cross_val_score(
    catboost_all_data_100,
    X_train,
    y_train,
    cv=StratifiedKFold(n_splits=4, shuffle=True, random_state=RANDOM_STATE),
    scoring='roc_auc',
    n_jobs=-1
)

cross_val_score_catboost_all_data_40 = cross_val_score(
    catboost_all_data_40,
    X_train,
    y_train,
    cv=StratifiedKFold(n_splits=4, shuffle=True, random_state=RANDOM_STATE),
    scoring='roc_auc',
    n_jobs=-1
)

cross_val_score_catboost_all_data_20 = cross_val_score(
    catboost_all_data_20,
    X_train,
    y_train,
    cv=StratifiedKFold(n_splits=4, shuffle=True, random_state=RANDOM_STATE),
    scoring='roc_auc',
    n_jobs=-1
)

print(f'{cross_val_score_catboost_all_data_100.mean()}'
      f'±{cross_val_score_catboost_all_data_100.std()}')

print(f'{cross_val_score_catboost_all_data_40.mean()}'
      f'±{cross_val_score_catboost_all_data_40.std()}')

print(f'{cross_val_score_catboost_all_data_20.mean()}'
      f'±{cross_val_score_catboost_all_data_20.std()}')

0.9195893124984393±0.0011780211076430536
0.9185903924815262±0.0011183835318481548
0.9170698947191096±0.001000244272439499


# Log Experiment Results 

In [26]:
auc_catboost_all_data_100 = roc_auc_score(
    y_test,
    catboost_all_data_100
    .predict_proba(
        cat_features_fillna(
            X_test[feature_importances_catboost_oversampling_relevant],
            set(cat_features).intersection(feature_importances_catboost_oversampling_relevant)
        )
    )[:, 1]
)

auc_catboost_all_data_40 = roc_auc_score(
    y_test,
    catboost_all_data_40
    .predict_proba(
        cat_features_fillna(
            X_test[feature_importances_catboost_oversampling_relevant],
            set(cat_features).intersection(feature_importances_catboost_oversampling_relevant)
        )
    )[:, 1]
)

auc_catboost_all_data_20 = roc_auc_score(
    y_test,
    catboost_all_data_20
    .predict_proba(
        cat_features_fillna(
            X_test[feature_importances_catboost_oversampling_relevant],
            set(cat_features).intersection(feature_importances_catboost_oversampling_relevant)
        )
    )[:, 1]
)

zip_run_info = (
    zip(['CatBoost all data 100 trees', 'CatBoost all data 40 trees', 'CatBoost all data 20 trees'],
        [catboost_all_data_100, catboost_all_data_40, catboost_all_data_20],
        ['catboost_all_data_100', 'catboost_all_data_40', 'catboost_all_data_20'],
        [auc_catboost_all_data_100, auc_catboost_all_data_40, auc_catboost_all_data_20])
)

Trackings = [
    Tracking(
        run_name=run_name,
        tags=dict(
            model_name=name,
            target=TARGET_CUSTOMER,
            num_features_used=len(model.feature_names_),
            top_feature_importances=(
                Counter(
                    dict(
                        zip(model.feature_names_,
                            model.feature_importances_)
                    )
                ).most_common(3)
            )
        ),
        params={param: value
                for param, value in (pipeline_catboost_oversampling
                                     .steps[-1][1]
                                     .get_params()
                                     .items())
                if param != 'cat_features'},
        metrics=dict(AUC=auc),
        model=model,
        model_name=name
    )
    for run_name, model, name, auc in zip_run_info
]

run_ids_mixed_data = (
    apply_runs_to_experiment(experiment_id_mixed_data,
                             Trackings)
)

# Kaggle Submission

# Which One Is The Best Model?

MLFlow allow us to get the best model within all combination of experiments we had run

We could try more parameters, algorithms and even data sets, but we have achieved a good result and it is important to have the model ready to be deployed

In [28]:
experiment_ids = [experiment_id_customers_vs_german_population,
                  experiment_id_training_data_provided,
                  experiment_id_mixed_data]


best_model_tracking = n_best_models_from_experiments(
    experiment_ids=experiment_ids, n=5, order_by=['metrics.AUC DESC']
)

best_model_tracking[['metrics.AUC', 'tags.mlflow.runName', 'params.num_trees', 'tags.num_features_used']]

,metrics.AUC,tags.mlflow.runName,params.num_trees,tags.num_features_used
0,0.769046,"Less features, oversampling with CatBoost",15,24
1,0.763107,"Less features, oversamplig, Hermert encoder wi...",15,1566
2,0.757247,Oversampling with CatBoost,15,365
3,0.753323,All possible features,None,365
4,0.752057,Less features,None,274


# Save Model Artifact Uri

We will use this artifact uri to make the model easy reproducible and deployable

In [29]:
best_model_artifact_uri = best_model_tracking['artifact_uri'].iloc[0]
best_model_model_name = best_model_tracking['tags.model_name'].iloc[0]

# Check MLFlow Model

In [30]:
model = load_trained_model(model_artifact_uri=best_model_artifact_uri,
                           model_model_name=best_model_model_name)

try:
    # This is model
    feature_importances_loaded_from_mlflow = dict(
        zip(model.feature_names_,
            model.feature_importances_)
    )
except AttributeError:
    # This is a pipeline
    feature_importances_loaded_from_mlflow = dict(
        zip(model.steps[-1][1].feature_names_,
            model.steps[-1][1].feature_importances_)
    )

feature_importances_loaded_from_mlflow

{'ANZ_KINDER': 0.20415072237784812,
 'D19_BANKEN_ANZ_12': 0.0837984349685739,
 'D19_BANKEN_ANZ_24': 0.4560739514707707,
 'D19_GESAMT_ONLINE_QUOTE_12': 0.9259713853611119,
 'D19_KONSUMTYP_MAX': 11.05821110969325,
 'D19_SOZIALES': 53.186382346877494,
 'D19_TELKO_REST': 1.1470836953730479,
 'EINGEFUEGT_AM': 2.2986116340657046,
 'EINGEZOGENAM_HH_JAHR': 10.646877982874827,
 'KBA05_HERST4': 2.648555184788434,
 'KBA05_KRSHERST2': 0.5631352580307166,
 'KBA05_KW2': 2.3691624662999216,
 'KBA05_MODTEMP': 1.5011674999932865,
 'KBA05_SEG1': 0.0010255933045166688,
 'KBA05_SEG2': 1.799807097504104,
 'KBA13_CCM_1800': 0.1998770352964555,
 'KBA13_KMH_110': 2.921490731720909,
 'KBA13_KMH_250': 2.878949128418511,
 'KBA13_VORB_D': 2.3630119506105243,
 'KK_KUNDENTYP': 0.0,
 'LP_LEBENSPHASE_GROB': 0.805456977952714,
 'SEMIO_FAM': 1.462451999813438,
 'SEMIO_PFLICHT': 0.0,
 'VERS_TYP': 0.47874781320380977}

# Conclusions

This was a very cool project very interesting to deal with it and to be able to apply different techniques

As future work I leave some points that would also be good to address:

- More exploration of variables and their knowledge
- Classic techniques as partial pooling
- Test other categorical encoders
- H2O models with MOJO approach to be deployable
- Sagemaker as a platform to implement and industrialize

## EXTRA (Just For Fun)

We're going to create an ensemble of all the predictions we've sent to kaggle

Bad result because we have very few sumbissions and for a good ensemble we need a lot non-correlated weak learners

In [31]:
df_ensemble_submissions = (
    pd
    .concat([pd.read_csv(file)
             for file in PATH_SUBMISSIONS.glob('*.csv')])
    .groupby('LNR')
    .agg(['mean', 'median'])
    .reset_index()
)

df_ensemble_submissions.columns = ['LNR', 'mean', 'median']

df_ensemble_submissions_mean = df_ensemble_submissions[['LNR', 'mean']].rename(columns=dict(mean=TARGET))
df_ensemble_submissions_median = df_ensemble_submissions[['LNR', 'median']].rename(columns=dict(median=TARGET))

kaggle_submission(
    column_lnr=df_ensemble_submissions_mean['LNR'],
    y_pred=df_ensemble_submissions_mean[TARGET],
    submission_filename='final_ensemble_mean',
    submission_message='Ensemble of all the Kaggle submissions with the mean'
)

kaggle_submission(
    column_lnr=df_ensemble_submissions_median['LNR'],
    y_pred=df_ensemble_submissions_median[TARGET],
    submission_filename='final_ensemble_median',
    submission_message='Ensemble of all the Kaggle submissions with the median'
)

100%|██████████| 1.04M/1.04M [00:05<00:00, 191kB/s]


Successfully submitted to Udacity+Arvato: Identify Customer Segments


100%|██████████| 1.05M/1.05M [00:06<00:00, 177kB/s]


Successfully submitted to Udacity+Arvato: Identify Customer Segments
